In [7]:
import asyncio
import time

In [2]:
async def main():
    print("Hello")
    await asyncio.sleep(5)
    print("world!")

In [5]:
await main()

Hello
world!


In [4]:
async def say_after(delay, what):
    await asyncio.sleep(delay)
    print(what)

In [16]:
def delay_say(delay, what):
    time.sleep(delay)
    print(what)

In [26]:
async def async_do_print():
    start_time = time.time()
    
    task1 = asyncio.create_task(say_after(4, 'Hello'))
    task2 = asyncio.create_task(say_after(5, 'World!'))
    
    await task1
    await task2
    
    duration = time.time() - start_time
    print(duration)

In [17]:
def do_print():
    start_time = time.time()
    
    delay_say(4, "Hello")
    delay_say(5, "world!")
    
    duration = time.time() - start_time
    print(duration)

In [18]:
do_print()

Hello
world!
9.006603717803955


In [27]:
await async_do_print()

Hello
World!
5.005457878112793


In [28]:
async def nested():
    return 42

In [29]:
nested()

<coroutine object nested at 0x110b05bc8>

In [30]:
await nested()

42

In [31]:
async def factorial(name, number):
    f = 1
    for i in range(2, number + 1):
        print(f"Task {name}: Compute factorial({i})...")
        await asyncio.sleep(1)
        f *= i
    print(f"Task {name}: factorial({number}) = {f}")

In [35]:
async def gathered_factorials():
    # Schedule three calls *concurrently*:
    await asyncio.gather(
        factorial("A", 3),
        factorial("B", 4),
        factorial("C", 5),
    )

In [40]:
start_time = time.time()
await gathered_factorials()
duration = time.time() - start_time
print(duration)

Task A: Compute factorial(2)...
Task B: Compute factorial(2)...
Task C: Compute factorial(2)...
Task A: Compute factorial(3)...
Task B: Compute factorial(3)...
Task C: Compute factorial(3)...
Task A: factorial(3) = 6
Task B: Compute factorial(4)...
Task C: Compute factorial(4)...
Task B: factorial(4) = 24
Task C: Compute factorial(5)...
Task C: factorial(5) = 120
4.007932901382446


In [41]:
start_time = time.time()
await factorial("A", 3)
await factorial("B", 4)
await factorial("C", 5)
duration = time.time() - start_time
print(duration)

Task A: Compute factorial(2)...
Task A: Compute factorial(3)...
Task A: factorial(3) = 6
Task B: Compute factorial(2)...
Task B: Compute factorial(3)...
Task B: Compute factorial(4)...
Task B: factorial(4) = 24
Task C: Compute factorial(2)...
Task C: Compute factorial(3)...
Task C: Compute factorial(4)...
Task C: Compute factorial(5)...
Task C: factorial(5) = 120
9.033505916595459


In [45]:
async def eternity():
    # Sleep for one hour
    await asyncio.sleep(4)
    print('yay!')

In [46]:
async def timeout_test():
    # Wait for at most 1 second
    try:
        await asyncio.wait_for(eternity(), timeout=1.0)
    except asyncio.TimeoutError:
        print('timeout!')

In [47]:
await timeout_test()

timeout!


In [57]:
async def foo():
    return 42

coro = foo()
done, pending = await asyncio.wait({coro})

if coro in done:
    print("HELLO!")
    
if coro in pending:
    print("PENDING!")

In [58]:
pending

set()

In [59]:
done

{<Task finished coro=<async-def-wrapper.<locals>.foo() done, defined at <ipython-input-57-7856e5915db9>:4> result=42>}

In [60]:
async def cancel_me():
    print('cancel_me(): before sleep')

    try:
        # Wait for 1 hour
        await asyncio.sleep(3600)
    except asyncio.CancelledError:
        print('cancel_me(): cancel sleep')
        raise
    finally:
        print('cancel_me(): after sleep')

In [62]:
async def cancel_me_main():
    # Create a "cancel_me" Task
    task = asyncio.create_task(cancel_me())

    # Wait for 1 second
    await asyncio.sleep(1)

    task.cancel()
    try:
        await task
    except asyncio.CancelledError:
        print("main(): cancel_me is cancelled now")

In [63]:
await cancel_me_main()

cancel_me(): before sleep
cancel_me(): cancel sleep
cancel_me(): after sleep
main(): cancel_me is cancelled now


In [64]:
async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def async_count():
    await asyncio.gather(count(), count(), count())

In [67]:
s = time.perf_counter()
await async_count()
elapsed = time.perf_counter() - s
print(f"Count executed in {elapsed:0.2f} seconds.")

One
One
One
Two
Two
Two
Count executed in 1.00 seconds.


In [72]:
import random

In [73]:
async def part1(n: int) -> str:
    i = random.randint(0, 10)
    print(f"part1({n}) sleeping for {i} seconds.")
    await asyncio.sleep(i)
    result = f"result{n}-1"
    print(f"Returning part1({n}) == {result}.")
    return result

async def part2(n: int, arg: str) -> str:
    i = random.randint(0, 10)
    print(f"part2{n, arg} sleeping for {i} seconds.")
    await asyncio.sleep(i)
    result = f"result{n}-2 derived from {arg}"
    print(f"Returning part2{n, arg} == {result}.")
    return result

async def chain(n: int) -> None:
    start = time.perf_counter()
    p1 = await part1(n)
    p2 = await part2(n, p1)
    end = time.perf_counter() - start
    print(f"-->Chained result{n} => {p2} (took {end:0.2f} seconds).")

async def main_chain():
    await asyncio.gather(*(chain(n) for n in [1, 2, 3]))

In [74]:
await main_chain()

part1(1) sleeping for 9 seconds.
part1(2) sleeping for 8 seconds.
part1(3) sleeping for 6 seconds.
Returning part1(3) == result3-1.
part2(3, 'result3-1') sleeping for 4 seconds.
Returning part1(2) == result2-1.
part2(2, 'result2-1') sleeping for 5 seconds.
Returning part1(1) == result1-1.
part2(1, 'result1-1') sleeping for 0 seconds.
Returning part2(1, 'result1-1') == result1-2 derived from result1-1.
-->Chained result1 => result1-2 derived from result1-1 (took 9.01 seconds).
Returning part2(3, 'result3-1') == result3-2 derived from result3-1.
-->Chained result3 => result3-2 derived from result3-1 (took 10.01 seconds).
Returning part2(2, 'result2-1') == result2-2 derived from result2-1.
-->Chained result2 => result2-2 derived from result2-1 (took 13.01 seconds).


In [75]:
import asyncio
import itertools as it
import os
import random
import time

async def makeitem(size: int = 5) -> str:
    return os.urandom(size).hex()

async def randsleep(a: int = 1, b: int = 5, caller=None) -> None:
    i = random.randint(0, 10)
    if caller:
        print(f"{caller} sleeping for {i} seconds.")
    await asyncio.sleep(i)

async def produce(name: int, q: asyncio.Queue) -> None:
    n = random.randint(0, 10)
    for _ in it.repeat(None, n):  # Synchronous loop for each single producer
        await randsleep(caller=f"Producer {name}")
        i = await makeitem()
        t = time.perf_counter()
        await q.put((i, t))
        print(f"Producer {name} added <{i}> to queue.")

async def consume(name: int, q: asyncio.Queue) -> None:
    while True:
        await randsleep(caller=f"Consumer {name}")
        i, t = await q.get()
        now = time.perf_counter()
        print(f"Consumer {name} got element <{i}>"
              f" in {now-t:0.5f} seconds.")
        q.task_done()

In [76]:
async def queue_main(nprod: int, ncon: int):
    q = asyncio.Queue()
    producers = [asyncio.create_task(produce(n, q)) for n in range(nprod)]
    consumers = [asyncio.create_task(consume(n, q)) for n in range(ncon)]
    await asyncio.gather(*producers)
    await q.join()  # Implicitly awaits consumers, too
    for c in consumers:
        c.cancel()

In [78]:
start = time.perf_counter()
await queue_main(2, 5)
elapsed = time.perf_counter() - start
print(f"Program completed in {elapsed:0.5f} seconds.")

Producer 0 sleeping for 4 seconds.
Consumer 0 sleeping for 4 seconds.
Consumer 1 sleeping for 7 seconds.
Consumer 2 sleeping for 4 seconds.
Consumer 3 sleeping for 4 seconds.
Consumer 4 sleeping for 8 seconds.
Producer 0 added <f058d7419b> to queue.
Producer 0 sleeping for 10 seconds.
Consumer 0 got element <f058d7419b> in 0.00033 seconds.
Consumer 0 sleeping for 7 seconds.
Producer 0 added <57e0c95af9> to queue.
Producer 0 sleeping for 8 seconds.
Consumer 2 got element <57e0c95af9> in 0.00046 seconds.
Consumer 2 sleeping for 4 seconds.
Producer 0 added <e48cf4c359> to queue.
Producer 0 sleeping for 7 seconds.
Consumer 3 got element <e48cf4c359> in 0.00036 seconds.
Consumer 3 sleeping for 1 seconds.
Producer 0 added <1b502b8233> to queue.
Consumer 1 got element <1b502b8233> in 0.00099 seconds.
Consumer 1 sleeping for 6 seconds.
Program completed in 29.01382 seconds.


In [79]:
async def mygen(u: int = 100):
    i = 0
    while i < u:
        yield i
        i += 1
        await asyncio.sleep(1.0)

In [80]:
g = [i async for i in mygen(10)]

In [1]:
import asyncio
import logging
import re
import sys
from typing import IO
import urllib.error
import urllib.parse

import aiofiles
import aiohttp
from aiohttp import ClientSession

In [11]:
session = ClientSession()
resp = await session.request(method="GET", url="https://realpython.com/")


In [14]:
resp.status

200

In [16]:
out = await resp.text()

In [20]:
import asyncio
import logging
import re
import sys
from typing import IO
import urllib.error
import urllib.parse

import aiofiles
import aiohttp
from aiohttp import ClientSession

logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(name)s: %(message)s",
    level=logging.DEBUG,
    datefmt="%H:%M:%S",
    stream=sys.stderr,
)
logger = logging.getLogger("areq")
logging.getLogger("chardet.charsetprober").disabled = True

HREF_RE = re.compile(r'href="(.*?)"')

async def fetch_html(url: str, session: ClientSession, **kwargs) -> str:
    """GET request wrapper to fetch page HTML.

    kwargs are passed to `session.request()`.
    """

    resp = await session.request(method="GET", url=url, **kwargs)
    resp.raise_for_status()
    logger.info("Got response [%s] for URL: %s", resp.status, url)
    html = await resp.text()
    return html

async def parse(url: str, session: ClientSession, **kwargs) -> set:
    """Find HREFs in the HTML of `url`."""
    found = set()
    try:
        html = await fetch_html(url=url, session=session, **kwargs)
    except (
        aiohttp.ClientError,
        aiohttp.http_exceptions.HttpProcessingError,
    ) as e:
        logger.error(
            "aiohttp exception for %s [%s]: %s",
            url,
            getattr(e, "status", None),
            getattr(e, "message", None),
        )
        return found
    except Exception as e:
        logger.exception(
            "Non-aiohttp exception occured:  %s", getattr(e, "__dict__", {})
        )
        return found
    else:
        for link in HREF_RE.findall(html):
            try:
                abslink = urllib.parse.urljoin(url, link)
            except (urllib.error.URLError, ValueError):
                logger.exception("Error parsing URL: %s", link)
                pass
            else:
                found.add(abslink)
        logger.info("Found %d links for %s", len(found), url)
        return found

async def write_one(file: IO, url: str, **kwargs) -> None:
    """Write the found HREFs from `url` to `file`."""
    res = await parse(url=url, **kwargs)
    if not res:
        return None
    async with aiofiles.open(file, "a") as f:
        for p in res:
            await f.write(f"{url}\t{p}\n")
        logger.info("Wrote results for source URL: %s", url)

async def bulk_crawl_and_write(file: IO, urls: set, **kwargs) -> None:
    """Crawl & write concurrently to `file` for multiple `urls`."""
    async with ClientSession() as session:
        tasks = []
        for url in urls:
            tasks.append(
                write_one(file=file, url=url, session=session, **kwargs)
            )
        await asyncio.gather(*tasks)

In [27]:
import pathlib
urls = set("https://realpython.com/")

await bulk_crawl_and_write(file="test.txt", urls=urls)

23:36:57 ERROR:areq: aiohttp exception for n [None]: None
23:36:57 ERROR:areq: aiohttp exception for t [None]: None
23:36:57 ERROR:areq: aiohttp exception for / [None]: None
23:36:57 ERROR:areq: aiohttp exception for . [None]: None
23:36:57 ERROR:areq: aiohttp exception for s [None]: None
23:36:57 ERROR:areq: aiohttp exception for e [None]: None
23:36:57 ERROR:areq: aiohttp exception for l [None]: None
23:36:57 ERROR:areq: aiohttp exception for y [None]: None
23:36:57 ERROR:areq: aiohttp exception for r [None]: None
23:36:57 ERROR:areq: aiohttp exception for c [None]: None
23:36:57 ERROR:areq: aiohttp exception for p [None]: None
23:36:57 ERROR:areq: aiohttp exception for h [None]: None
23:36:57 ERROR:areq: aiohttp exception for : [None]: None
23:36:57 ERROR:areq: aiohttp exception for a [None]: None
23:36:57 ERROR:areq: aiohttp exception for m [None]: None
23:36:57 ERROR:areq: aiohttp exception for o [None]: None
